In this notebook, we have implemented a heuristic to check whether a matroid M is Frobenius-flock representable.
One by one, we enumerate the combinatorial types of M, pick an integral valuation v of this combinatorial type, and check whether M^v is Frobenius-flock representable. This continues until a Frobenius-flock representable matroid flock is found, or all combinatorial types have been enumerated. Depending on the outcome, conclusions are drawn about Frobenius-flock representability of M: True, False or Maybe.

In [1]:
def obtainU24Bases(S,T):
    #Compute the bases of the U24-minor (M/S)|_T
    t = set(T)
    a = t.pop()
    b = t.pop()
    c = t.pop()
    d = t.pop()
    B11 = frozenset(S.union([a,b]))
    B12 = frozenset(S.union([c,d]))
    B21 = frozenset(S.union([a,c]))
    B22 = frozenset(S.union([b,d]))
    B31 = frozenset(S.union([a,d]))
    B32 = frozenset(S.union([b,c]))
    return B11,B12,B21,B22,B31,B32

def u24minors(M):
    #computes all U24-minors of M
    U24M = []
    E = M.groundset()
    r = M.full_rank()
    if r<2:
        return []
    bases = M.bases()
    for U in Subsets(E, 4):
        U2 = frozenset(U)
        U = set(U)
        a = U.pop()
        b = U.pop()
        c = U.pop()
        d = U.pop()
        for S in Subsets(E-U, r-2):
            S = frozenset(S)
            B = S.union({a,b})
            B2= S.union([c,d])
            if not(B in bases and B2 in bases):
                continue    
            B = S.union([a,c])
            B2= S.union([b,d])
            if not(B in bases and B2 in bases):
                continue    
            B = S.union([a,d])
            B2= S.union([b,c])
            if not (B in bases and B2 in bases):
                continue
            #U24 minor
            U24M.append([obtainU24Bases(S,U2),[0,1,2,3]])
    return U24M

def initialConstraints(M):
    #Compute the constraints from the degenerate pure quadrangles of M
    B=list(M.bases())
    R=[]
    R2=[]
    for X in M.nonbases():
        if M.rank(X)<M.full_rank()-1:
            continue
        C=set(M.circuit(X))
        D=set(M.cocircuit(M.groundset()-X))
        e = C.pop()
        f = D.pop()
        for e2 in C:
            for f2 in D:
                r={frozenset(X).symmetric_difference([e,f]):1, 
                frozenset(X).symmetric_difference([e,f2]):-1,
                frozenset(X).symmetric_difference([e2,f]):-1, 
                frozenset(X).symmetric_difference([e2,f2]):1}
                R.append(r)
                R2.append(r)
    for e in M.groundset():
        r = {b: 1 for b in B if e in b }
        R.append(r)                  

    idx = {B[i]: i for i in range(len(B))}
    C = matrix(QQ, len(R2), len(B))
    for i in range(len(R2)):
        for b, a in R2[i].items():
            C[i, idx[b]] = a
    
    piv = C.pivot_rows()
    delset = list(set(range(C.nrows())) - set(piv))
    C = C.delete_rows(delset)
    
    constraints=[]
    
    for rw in C.rows():
        summ=0
        for i in range(len(B)):
            summ += rw[i]*BB[frozenset(B[i])]
        constraints.append(summ == 0)
    
    return constraints

mip=None
BB=None
def initializeMip(M):
    global mip
    global BB
    mip = MixedIntegerLinearProgram(maximization=False)
    BB = mip.new_variable(nonnegative=True,real=True)
    for con in initialConstraints(M):
        mip.add_constraint(con)

def mipRemoveConstraints(n):
    if n>0:
        nc = mip.number_of_constraints()
        mip.remove_constraints([nc-1-i for i in range(n)])

def mipAddConstraints(u,c):
    con1,con2 = U24Constraints(u,c)
    mip.add_constraint(con1)
    mip.add_constraint(con2)

def U24Constraints(u,c):
    B11,B12,B21,B22,B31,B32 = u
    if c == 3:
        return [BB[B11] + BB[B12] == BB[B21] + BB[B22], BB[B11] + BB[B12] <= BB[B31] + BB[B32] - 1]
    elif c == 2:
        return [BB[B11] + BB[B12] == BB[B31] + BB[B32], BB[B11] + BB[B12] <= BB[B21] + BB[B22] - 1]
    elif c == 1:
        return [BB[B31] + BB[B32] == BB[B21] + BB[B22], BB[B31] + BB[B32] <= BB[B11] + BB[B12] - 1]
    elif c == 0:
        return [BB[B31] + BB[B32] == BB[B21] + BB[B22], BB[B31] + BB[B32] == BB[B11] + BB[B12]]

def updateUAvailable(U):
    newU = []
    nextu = None
    maxlen = 0
    nAddedConstraints = 0
    
    for u in U:
        availableCases = u[1]
        newAvailableCases = []
        
        for c in availableCases:
            if isFeasible(u[0],c):
                newAvailableCases.append(c)
        
        if len(newAvailableCases) == 0:
            return "INFEASIBLE", nAddedConstraints
        
        elif len(newAvailableCases) == 1:
            mipAddConstraints(u[0],newAvailableCases[0])
            nAddedConstraints += 2
            
        else:
            newU.append([u[0],newAvailableCases])
            l = len(newAvailableCases)
            if l > maxlen:
                nextu = newU[-1]
                maxlen = l
    return newU, nextu, nAddedConstraints

flockRepresentable = None

def Solve(p):
    global flockRepresentable
    mip.set_integer(BB)
    try:
        mip.solve()
        valuation = mip.get_values(BB)
        mip.set_real(BB)
        
    except:
        mip.set_real(BB)
        return [None,[False,"False"]]    
        
    valuation = {b:ZZ(valuation[b]) for b in valuation}
    
    flockRep = isFFlockRepresentable(valuation,p)
    
    if flockRep[0]:
        flockRepresentable = "True"
    elif flockRep[1].find("groebner") != -1:
        flockRepresentable = "Maybe"
            
    return valuation, flockRep
    
def isFeasible(u,c):
    mipAddConstraints(u,c)
        
    feasible=None
    try:
        mip.solve(objective_only=True)
        feasible=True
    except:
        feasible=False
    
    nc=mip.number_of_constraints()
    mipRemoveConstraints(2)
    
    return feasible

def Branch(U,p):
    if flockRepresentable == "True":
        return []
    valuations = None
    update = updateUAvailable(U)
    if update[0] == "INFEASIBLE":
        mipRemoveConstraints(update[1])
        return []
    else:
        newU, u, nImpliedConstraints = update
        if len(newU) > 0:
            cases = u[1]
            newU.remove(u)
            valuations = []
            for c in cases:
                mipAddConstraints(u[0],c)
                valuations += Branch(newU,p)
                mipRemoveConstraints(2)
        else:
            valuations = []
            flock = Solve(p)
            if flock[1][0] or flock[1][1] == "groebner":
                valuations.append([flock[0],flock[1][0]])
            
        mipRemoveConstraints(nImpliedConstraints)
        return valuations

    
    
    
    
    
    
    
    
    
class MatroidSet():
    def __init__(self):
        self._R={}
        self._mats={}

    def __repr__(self):
        return str(len(self))+' matroids'
    
    def __contains__(self, M):
        q=M._bases_invariant()
        if q in self._R:
            for ni in self._R[q]:
                if M.is_isomorphic(self._matroidOf(ni)):
                    return True
        return False
                 
    def __len__(self):
         return sum([len(v) for v in self._R.values()])
         
    def list(self):
        return flatten(self._R.values())
    
    def __iter__(self):
        for ML in self._R.values():
            for M in ML:
                yield M
    
    def _matroidOf(self,mi):
        return self._mats[mi]
    
    def findIndex(self, M):
        q=M._bases_invariant()
        if q in self._R:
            for ni in self._R[q]:
                if M.is_isomorphic(self._matroidOf(ni)):
                    return ni
        return None
                
    def add(self, mi, M):    
        q=M._bases_invariant()
        if q in self._R:
            found=False
            if mi not in self._R[q]:
                self._R[q].append(mi)
        else:
            self._R[q]=[mi]
        self._mats[mi]=M
        
def minDegBasis(m):
    #Compute a basis with the fewest neighbors in the base exchange graph
    basis = []
    neighborsB = []
    neighbors = 10000
    for b in m.bases():
        nB = []
        for e in b:
            for f in m.groundset().difference(b):
                if m.is_basis(b.difference({e}).union({f})):
                    nB.append([e,f])
        l = len(neighborsB)
        if l < neighbors:
            neighbors = l
            neighborsB = nB
            basis = b
    return basis, neighborsB

def complement(S,EE):
    E = {EE[i] : i for i in range(len(EE))}
    return [E[s] for s in EE if s not in S]

def Balpha(M, val, alpha):
    #Compute the set B^val_alpha
    val2={ b: sum([alpha[i] for i in b])-val[b] for b in val}
    m = max(val2.values())
    Balpha=[b for b in val.keys() if val2[b]==m]
    return Balpha

def Malpha(M, val, alpha):
    #Compute the matroid M^val_alpha
    E= M.groundset()
    B=Balpha(M,val, alpha)
    return BasisMatroid(groundset=E, bases=B)

def centralPoints(val,M = None, dic=True, integral=True):
    lp = MixedIntegerLinearProgram()
    alpha = lp.new_variable()
    if not M:
        M = BasisMatroid(Matroid(bases = [B for B in val]))
    E = M.groundset()
    
    #fix the order of the variables in lp by adding dummy constraints
    for e in E:
        lp.add_constraint(alpha[e]==0)
        
    #add the constraints
    tau = {}
    for e in E:
        tau[e] = {B:1*(e in B) for B in val}
    rho = {}
    for B in val:
        rho[B] = val[B] - sum([alpha[e]*tau[e][B] for e in E])
        lp.add_constraint(rho[B] >= 0)
    
    #remove dummy constraints:
    lp.remove_constraints(range(len(E)))
    
    #the central points are the vertices of the polyhedron of the linear program
    pts = lp.polyhedron().vertices_list()
    QQpts = [[QQ(v) for v in pt] for pt in pts]
    
    Ei = {}
    i = 0
    for e in E:
        Ei[e] = i
        i += 1
    
    #normalize points: smallest value 0 in each component
    normPts = []
    normPtsDict = []
    comp = [J for J in M.components()]
    indices = {J:sorted([Ei[j] for j in J]) for J in comp}
    for pt in QQpts:
        mins = {}
        for J in comp:
            minJ = min([pt[j] for j in indices[J]])
            for e in J:
                mins[e] = minJ
        
        #when the input valuation is integral, the central points are integral;
        #we round to counteract machine errors
        if integral:
            normPts.append([round(pt[Ei[e]] - mins[e]) for e in E])
            normPtsDict.append({e:round(pt[Ei[e]] - mins[e]) for e in E})
        else:
            normPts.append([pt[Ei[e]] - mins[e] for e in E])
            normPtsDict.append({e:pt[Ei[e]] - mins[e] for e in E})
    
    if dic:
        return normPtsDict
    else:
        return normPts


def constructMatrix(M,F,d=0,pointZero=False,verbose=False):
    #d is the index of the first variable x_d
    rk = M.rank()
    n = M.size()
    B, neighborsB = minDegBasis(M)
    
    T = []
    if pointZero:
        G=Graph(neighborsB)
        for J in G.connected_components():
            H = G.subgraph(J)
            T.extend([v[0:2] for v in H.min_spanning_tree()])
    
    X = ['x%d' % i for i in range(d,d + len(neighborsB) - len(T))]
    
    R = PolynomialRing(F,X)
    if len(X) == 0:
        R = F
    
    E = M.groundset()
    columns = {}
    j=0
    d0 = 0
    for e in E:
        if e in B:
            #create Identity matrix at B
            columns[e] = vector([(i==j)*F(1) for i in range(len(B))])
            j+=1
    for e in E:
        if e not in B:
            C = M.fundamental_circuit(B,e)
            columns[e] = vector([F(0) for _ in range(len(B))])
            for f in C.difference({e}):
                if pointZero and ((e,f) in T or (f,e) in T):
                    columns[e] = columns[e] + columns[f]
                else:
                    columns[e] = columns[e] + R('x%d' % (d+d0))*columns[f]
                    d0+=1
    Xnew = [('x%d' % i) for i in range(d,d+d0)]
    
    rep = matrix(R,[[R(a) for a in list(columns[e])] for e in E]).transpose()
    
    #if rep = [I A], then its dual is [-A]
    #                                 [ I]
    
    A = matrix(R,[[R(a) for a in list(columns[e])] for e in E if not e in B]).transpose()
    
    dualrows = {}
    j=0
    for e in E:
        if e in B:
            dualrows[e] = -A[j]
            j+=1
    j=0
    for e in E:
        if not e in B:
            dualrows[e] = vector([(i==j)*F(1) for i in range(len(E)-len(B))])
            j+=1
    
    dualrep = matrix(R,[[R(a) for a in list(dualrows[e])] for e in E])
    
    return rep, dualrep, d0
    
    
    


def findMatroidEquations(Ma,rep,d=0):
    eqs = []
    E = [e for e in Ma.groundset()]
    R = rep.base_ring()
    d0 = 0
    if R.ngens() == 0: 
        #(in order to prevent buggy behavior)
        R = R.base_ring()
    for nb in Ma.nonbases():
        D = rep.delete_columns(complement(nb,E)).determinant()
        if D != 0:
            eqs.append(D)
    for b in Ma.bases():
        R2 = PolynomialRing(R,['t%d' % (d+d0)])
        D = rep.delete_columns(complement(b,E)).determinant()
        if D not in R.base_ring() or D == 0:
            eqs.append(1 - R2('t%d' % (d+d0))*R(D))
            d0 += 1
        
    return eqs, d0


def neighboringPairs(E,centralMatroids):
    cp = {pt:vector(ZZ,pt) for pt in centralMatroids}
    edges = []
    for pair in Subsets([pt for pt in centralMatroids],2):
        p1,p2 = pair
        v = cp[p1] - cp[p2]
        dif = set(v)
        if len(dif)!=2:
            continue
        
        #p1 and p2 might be neighbors:
        l = min(dif)
        s = max(dif)
        k = s-l
        #p1 = p2 + (s-l) e_I  + l*1
        #p2 = p1 + (s-l) e_Ic + s*1
        
        EI = [E[e] for e in range(len(E)) if v[e] == s]
        EIc = [E[e] for e in range(len(E)) if v[e] == l]
        
        Mi = centralMatroids[p1]
        Mj = centralMatroids[p2]
        
        if Mj.contract(EI) != Mi.delete(EI) or Mi.contract(EIc) != Mj.delete(EIc):
            continue
            
        if abs(l) < abs(s):
            yield [p2,p1], k, l, EI
        else:
            yield [p1,p2], k, s, EIc


def findNeighborEquations(rep1,rep2,k,l,EI,E, p):
    m1, dm1 = rep1
    m2, dm2 = rep2
    I = [i for i in range(len(E)) if E[i] in EI]
    Ic= [i for i in range(len(E)) if not E[i] in EI]
    
    #p2 = p1 + k e_I + l*1 = p1 + (k+l) e_I + l e_Ic
    
    #(FF1):      m1/I  = m2\I
    #       -k*1 m2/Ic = m1\Ic 
    
    eqs = []
    
    if l >= 0:
        l1dm1 = matrix([[x^(p^l) for x in v] for v in dm1])
        eqs += (m2.delete_columns(I) * l1dm1.delete_rows(I)).list()
    else:
        l1m2 = matrix([[x^(p^(-l)) for x in v] for v in m2])
        eqs += (l1m2.delete_columns(I) * dm1.delete_rows(I)).list()
        
    if k+l >= 0:    
        kl1m1 = matrix([[x^(p^(k+l)) for x in v] for v in m1])
        eqs += (kl1m1.delete_columns(Ic) * dm2.delete_rows(Ic)).list()
    else:
        kl1dm2 = matrix([[x^(p^(-k-l)) for x in v] for v in dm2])
        eqs += (m1.delete_columns(Ic) * kl1dm2.delete_rows(Ic)).list()
    
    return [eq for eq in eqs if eq != 0]



def dualValuation(M,val):
    E = M.groundset()
    return {E-b:val[b] for b in val}

def valDelete(M,val,J):
    r = M.delete(J).full_rank()
    return {b-J:val[b] for b in val if len(b-J) == r}



from sage.matroids.advanced import *
def isFFlockRepresentable(val,p,M=False):
    if len(val) == 0:
        return True, "trivial"
    
    if not M:
        M = Matroid(bases=[B for B in val])
        
    EE = M.groundset()
    
    if len(EE) <= 3:
        #all small enough matroids are rigid and linear, so each valuation is trivial and flock-representable
        return True, "size" 
    
    linear = isLinearMatroid(M,p)
    if linear != "unknown" and linear:
        #the trivial valuation is representable
        return True, "linearity"
    
    val = dualValuation(M,val)
    #the remainder of the procedure computes F^-1-flock representability,
    #which is equivalent to F-flock representability of the dual
    
    F = GF(p)
    E = list(EE)
    n = len(E)
    idx = {E[i]:i for i in range(n)}
    
    cp = centralPoints(val,M)
    centralPts = [tuple([pt[e] for e in E]) for pt in cp]
    centralMatroids = {centralPts[i]:Malpha(M,val,cp[i]) for i in range(len(centralPts))}
    
    
    #first check linearity of the central points
    if linear != "unknown":
        for pt in centralPts:
            linear = isLinearMatroid(centralMatroids[pt],p)
            if not linear:
                return False, "linearity"

    matrices = {}
    dualMatrices = {}
    d = 0
    
    matroidEquations = []
    d2 = 0
    
    for pt in centralPts:
        zero=False
        if len(set(pt)) == 1:
            #the zero point
            zero=True
        rep, dualrep, dInc = constructMatrix(centralMatroids[pt],F,d, pointZero=zero)
        matrices[pt] = rep
        dualMatrices[pt] = dualrep
        d += dInc
        
        eqs, d2Inc = findMatroidEquations(centralMatroids[pt], matrices[pt], d2)
        matroidEquations += eqs
        d2 += d2Inc
    
    X = ['x%d' % i for i in range(d)]
    T = ['t%d' % j for j in range(d2)]
    R = PolynomialRing(F,X)
    
    for pt in centralPts:
        matrices[pt] = matrices[pt].change_ring(R)
        dualMatrices[pt] = dualMatrices[pt].change_ring(R)
    
    neighborEquations = []
        
    for pair, k, l, EI in neighboringPairs(E,centralMatroids):
        p1,p2 = pair
        eqs = findNeighborEquations([matrices[p1],dualMatrices[p1]], [matrices[p2],dualMatrices[p2]], k, l, EI, E, p)
        neighborEquations += eqs
    
    S = PolynomialRing(F,X+T)
    equations = [S(eq) for eq in matroidEquations] + [S(eq) for eq in neighborEquations]
    
    id = S.ideal(equations)
    gb = id.groebner_basis('libsingular:groebner',prot=False)
    
    if 1 in gb:
        return False, "groebner"
    else:
        return True, "groebner"



from sage.matroids.advanced import *
def matroidIsFFlockRepresentable(M,p):
    #Computes one valuation per combinatorial type, until a Frobenius-flock representable valuation is found.
    #-If one of the valuations is Frobenius-flock representable, returns this valuation, and 
    #   sets FlockRepresentable to "True".
    #-If all valuations are not Frobenius-flock representable due to nonlinearity of the M^val_alpha, 
    #  returns [] and sets FlockRepresentable to "False".
    #-If all valuations are not Frobenius-flock representable, but not all due to nonlinearity of any of the M^val_alpha, 
    #  returns the list of valuations for which this is not due to nonlinearity, and sets FlockRepresentable to "Maybe".
    
    global flockRepresentable
    
    if not M.is_connected():
        EE = M.groundset()
        J = M.components()[0]
        I = EE-J
        vals1 = matroidIsFFlockRepresentable(M.delete(I),p)
        FR1 = flockRepresentable
        vals2 = matroidIsFFlockRepresentable(M.delete(J),p)
        FR2 = flockRepresentable
        
        flockRepresentable = FR1 and FR2
        
        return vals1 + vals2
    
    U = u24minors(M)
    initializeMip(M)
    flockRepresentable = "False"
    return Branch(U,p)
    
    
    
def skeletonGraph(M,val):
    #shows the skeleton graph of M^val (except the rays)
    E = M.groundset()
    cp = centralPoints(val,M)
    centralPts = [tuple([pt[e] for e in E]) for pt in cp]
    centralMatroids = {centralPts[i]:Malpha(M,val,cp[i]) for i in range(len(cp))}
    neighbors = neighboringPairs(list(E),centralMatroids)
    edges = [N[0] for N in neighbors]
    return Graph(edges)

In [2]:
#In order to check linearity of matroids, we look the matroids up in a matroid set of nonlinear matroids.
#Edit the path to match your own directories.

path = "/media/sf_SageShared/fromScratch/non2linearMatroidSet.sobj"



non2linearMS = load(path)
def isLinearMatroid(M,p):
    n = M.size()
    r = M.full_rank()
    k = (r,n)
    kd = (n-r,n)
    
    if p == 2:
        if k in non2linearMS:
            return not M in non2linearMS[k]
        if kd in non2linearMS:
            return not M.dual() in non2linearMS[kd]
        return True
    else:
        return "unknown"

Examples:

In [4]:
%%time
matroidIsFFlockRepresentable(matroids.named_matroids.NonFano(),2)
print flockRepresentable

True
CPU times: user 208 ms, sys: 8.45 ms, total: 216 ms
Wall time: 484 ms


In [5]:
%%time
matroidIsFFlockRepresentable(matroids.named_matroids.Block_9_4(),2)
print flockRepresentable

False
CPU times: user 1.47 s, sys: 4.19 ms, total: 1.48 s
Wall time: 1.5 s
